In [1]:
import pandas as pd
import numpy as np

In [29]:
Exp_df=pd.read_csv('Expp-1.csv')

In [30]:
descsss=[]
for i in range(Exp_df.shape[0]):
    descsss.append(Exp_df['Final_desc'][i])

In [32]:
input_string='''Smart Interns who possess strong technical development skills and knowledge of advanced topics in Data Science, Machine Learning & Artificial IntelligenceAn ideal candidate would be An individual who is passionate about Data Science, Machine Learning & Artificial Intelligence and build content in this spaceResponsible for building quality content - social posts and blogs in Data Science Moderate a vibrant Data Science community'''
input_string

'Smart Interns who possess strong technical development skills and knowledge of advanced topics in Data Science, Machine Learning & Artificial IntelligenceAn ideal candidate would be An individual who is passionate about Data Science, Machine Learning & Artificial Intelligence and build content in this spaceResponsible for building quality content - social posts and blogs in Data Science Moderate a vibrant Data Science community'

In [33]:
import nltk,string
from nltk.corpus import stopwords

puncs=string.punctuation

stops=stopwords.words('english')

from re import sub
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

In [34]:
# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)
#print(similarity_index)

query_string = input_string


# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stops and token not in puncs]

In [35]:
# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in descsss]
query = preprocess(query_string)

In [36]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
#print(dictionary.token2id)
tfidf = TfidfModel(dictionary=dictionary)
#print(tfidf)
# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)
#print(similarity_matrix)
query_tf = tfidf[dictionary.doc2bow(query)]
#print(query_tf)
index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)
doc_similarity_scores = index[query_tf]

[(15, 0.1196984779992493), (26, 0.0894406090885117), (37, 0.18245768556515124), (42, 0.18245768556515124), (63, 0.28411726054275444), (67, 0.09122884278257562), (93, 0.1196984779992493), (94, 0.09122884278257562), (117, 0.07102931513568861), (215, 0.1196984779992493), (223, 0.1196984779992493), (267, 0.1196984779992493), (369, 0.1196984779992493), (396, 0.33673528172562), (397, 0.16836764086281), (398, 0.16836764086281), (399, 0.16836764086281), (400, 0.33673528172562), (401, 0.16836764086281), (402, 0.16836764086281), (403, 0.16836764086281), (404, 0.16836764086281), (405, 0.16836764086281), (406, 0.16836764086281), (407, 0.16836764086281), (408, 0.16836764086281), (409, 0.16836764086281), (410, 0.16836764086281), (411, 0.16836764086281), (412, 0.16836764086281), (413, 0.16836764086281), (414, 0.16836764086281), (415, 0.16836764086281)]


In [39]:
doc_similarity_scores=doc_similarity_scores*100

In [46]:
doc_similarity_scores=[round(i) for i in doc_similarity_scores]    

In [47]:
Exp_df['Rank']=doc_similarity_scores

In [49]:
Exp_df.to_csv('Final_data.csv')